# 🚀 Подггтовка BERT для CRF

---

## 📦 1. Установка зависимостей

In [1]:
!pip install pyspellchecker pymorphy3 rapidfuzz evaluate seqeval fuzzywuzzy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 116.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 79.2 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=dd5efd9c184dc8316c60726e79d057ee6213a6015c460eed1814f29e0dd6fe0e
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [2]:
!pip install --upgrade accelerate

In [3]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 127.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [4]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/lct_2025/')

Mounted at /content/drive


In [5]:
import numpy as np
import pandas as pd
import random
import re
import json
import asyncio
import pymorphy3
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, pipeline
from transformers import BertConfig, BertForTokenClassification, BertTokenizerFast, pipeline, BertModel
from transformers import EarlyStoppingCallback
from datasets import Dataset
from sklearn.metrics.pairwise import cosine_similarity
import torch
from sklearn.preprocessing import normalize
from fuzzywuzzy import process
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score, precision_score, recall_score
from ast import literal_eval
import evaluate
from functools import partial

/usr/local/lib/python3.12/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [6]:
# Импорты
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import AdamW
import re
import ast
import random
import json
import os
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Transformers
from transformers import (
    AutoTokenizer, AutoModel, AutoConfig,
    Trainer, TrainingArguments,
    DataCollatorForTokenClassification,
    get_linear_schedule_with_warmup
)

# Datasets и метрики
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
import evaluate

# Настройка для воспроизводимости
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# Проверка GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## ⚙️ 2. Конфигурация и паттерны

In [7]:
class Config:
    """Конфигурация модели с расширенными паттернами для VOLUME/PERCENT."""

    # Модель и токенизатор
    MODEL_NAME = 'cointegrated/rubert-tiny2'# 'cointegrated/rubert-tiny2'  # Быстрая русская модель
    MAX_LENGTH = 128
    BATCH_SIZE = 32  # Увеличиваем для Colab GPU
    OUTPUT_DIR = './tiny2_v6'

    # Параметры обучения
    EPOCHS = 15
    LEARNING_RATE = 3e-5
    CRF_LEARNING_RATE = 1e-4
    WEIGHT_DECAY = 0.01
    WARMUP_RATIO = 0.1
    GRADIENT_ACCUMULATION_STEPS = 1

    # Метки (только TYPE, BRAND, O для обучения)
    LABELS = ['O', 'B-TYPE', 'I-TYPE', 'B-BRAND', 'I-BRAND']
    LABEL_TO_ID = {label: idx for idx, label in enumerate(LABELS)}
    ID_TO_LABEL = {idx: label for label, idx in LABEL_TO_ID.items()}

    # 🔍 Расширенные VOLUME паттерны с полными склонениями и ошибками
    VOLUME_PATTERNS = [
        # ЛИТРЫ - все падежи и варианты написания
        r'\b(\d+(?:[.,]\d+)?)\s*(?:литр|литра|литров|литре|литрам|литрами|литрах|литру|литром|л|l)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:литер|литера|литеров|литеры|литеру|литером|литере|литерам|литерами)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:лит|лита|литы|литов|литу|литом)\b',

        # ГРАММЫ - все падежи и варианты с ошибками
        r'\b(\d+(?:[.,]\d+)?)\s*(?:г|гр|грамм|грамма|граммов|граммы|грамму|граммом|грамме|граммам|граммами|граммах)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:грам|грама|грамов|грамы|граму|грамом|граме|грамам|грамами|грамах)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:грм|грма|грмов|грмы|грму|грмом|грме)\b',

        # КИЛОГРАММЫ - все падежи и варианты с ошибками
        r'\b(\d+(?:[.,]\d+)?)\s*(?:кг|килограмм|килограмма|килограммов|килограммы|килограмму|килограммом|килограмме|килограммам|килограммами|килограммах)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:килаграм|килаграмма|килаграммов|килаграммы|килаграмму|килаграммом|килаграмме)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:киллограм|киллограмма|киллограммов|киллограммы)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:кило|килло)\b',

        # МИЛЛИЛИТРЫ - все падежи и ошибки
        r'\b(\d+(?:[.,]\d+)?)\s*(?:мл|ml|миллилитр|миллилитра|миллилитров|миллилитры|миллилитру|миллилитром|миллилитре)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:милитр|милитра|милитров|милитры|милитру|милитром|милитре)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:милилитр|милилитра|милилитров|милилитры)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:милл|мила)\b',

        # ШТУКИ и УПАКОВКИ с ошибками
        r'\b(\d+(?:[.,]\d+)?)\s*(?:шт|штук|штука|штуки|штуку|штукой|штуке|штукам|штуками|штуках)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:штукы|штукы|штуку|штукой)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:упак|упаковк|упаковка|упаковки|упаковок|упаковку|упаковкой|упаковке)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:упоковка|упоковки|упоковок)\b',

        # ДОПОЛНИТЕЛЬНЫЕ форматы
        r'\b(\d+(?:[.,]\d+)?)\s*[x×х]\s*(\d+(?:[.,]\d+)?)\b',
        r'\b(\d+/\d+|\d+,\d+|\d+\.\d+)\s*(?:л|литр|мл|г|грамм|кг|килограмм)\b'
    ]

    # 📈 Расширенные PERCENT паттерны с учетом ошибок
    PERCENT_PATTERNS = [
        r'\b(\d+(?:[.,]\d+)?)\s*%',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:процент|процента|процентов|процентный|процентная|процентное)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:проц|процентов|прцент|прцентов|прцент)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:жирн|жирност|жирности|жирностью|жирностей)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:жырн|жырност|жырности)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:град|градус|градуса|градусов|градусы|°|гр\.)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:грдус|грдуса|грдусов|грдусы)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:алк|алкоголь|алкогольный|алкогольная|алкогольное)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:алког|алкогол|алкогольны)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:концентрац|концентрация|концентрации|концентрацией|концентраций)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:канцентрац|канцентрация|канцентрации)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:содержан|содержание|содержания|содержанием|содержаний)\b'
    ]

    # Standalone числа
    STANDALONE_NUMBER_PATTERN = r'\b(\d+(?:[.,]\d+)?)\b'

    # Исключения для standalone чисел
    EXCLUDE_PATTERNS = [
        r'\b(19\d{2}|20\d{2})\b',  # года
        r'\b\d{4}\s*(?:год|года|годов|году|годом|годе)\b',
        r'\b(?:номер|код|артикул|арт|№)\s*\d+\b',  # номера
        r'\b\d+\s*(?:руб|рубл|рублей|рубля|р\.|₽|долл|доллар|\$|€|евро)\b',  # цены
        r'\b\d{1,2}:\d{2}\b',  # время
        r'\b\d{1,2}[./]\d{1,2}[./]\d{2,4}\b'  # даты
    ]

config = Config()

def is_standalone_volume(text, number_match):
    """Проверяет, является ли отдельно стоящее число VOLUME сущностью."""
    number_start, number_end = number_match.span()
    number_text = number_match.group()

    # Проверяем исключения
    for exclude_pattern in config.EXCLUDE_PATTERNS:
        for exclude_match in re.finditer(exclude_pattern, text, re.IGNORECASE):
            if exclude_match.start() <= number_start < exclude_match.end():
                return False

    # Проверяем контекст (в радиусе 15 символов)
    context_start = max(0, number_start - 15)
    context_end = min(len(text), number_end + 15)
    context = text[context_start:context_end]

    # Проверяем VOLUME/PERCENT паттерны в контексте
    for pattern in config.VOLUME_PATTERNS + config.PERCENT_PATTERNS:
        if re.search(pattern, context, re.IGNORECASE):
            return False

    # Проверяем размер числа
    try:
        num_value = float(number_text.replace(',', '.'))
        if num_value > 10000 or num_value == 0:
            return False
    except ValueError:
        pass

    return True

print("✅ Конфигурация загружена!")
print(f"📊 Метки для обучения: {config.LABELS}")
print(f"🔍 VOLUME паттернов: {len(config.VOLUME_PATTERNS)}")
print(f"📈 PERCENT паттернов: {len(config.PERCENT_PATTERNS)}")
print(f"⚙️ Batch size: {config.BATCH_SIZE}")
print(f"🎯 Цель: macro-F1 ≥ 0.8")

✅ Конфигурация загружена!
📊 Метки для обучения: ['O', 'B-TYPE', 'I-TYPE', 'B-BRAND', 'I-BRAND']
🔍 VOLUME паттернов: 20
📈 PERCENT паттернов: 12
⚙️ Batch size: 32
🎯 Цель: macro-F1 ≥ 0.8


## 📂 3. Загрузка и обработка данных



In [8]:
# объединение спарсенных данных
'''files = os.listdir('./parsed_data')
data = pd.read_csv(f'./parsed_data/{files[0]}', sep=';')['product-name']
for f in files[1:]:
    data = pd.concat([data, pd.read_csv(f'./parsed_data/{f}', sep=';')['product-name']], ignore_index=True)
data.to_csv('parsed_data.csv', index=False, sep=';')'''

"files = os.listdir('./parsed_data')\ndata = pd.read_csv(f'./parsed_data/{files[0]}', sep=';')['product-name']\nfor f in files[1:]:\n    data = pd.concat([data, pd.read_csv(f'./parsed_data/{f}', sep=';')['product-name']], ignore_index=True)\ndata.to_csv('parsed_data.csv', index=False, sep=';')"

In [9]:
# добавление ручной разметки
'''data = pd.read_csv(f'./train_big.csv', sep=';')
new = pd.read_csv(f'./my_data.csv', sep=';')
data = pd.concat([data, new], ignore_index=True)
data.to_csv('train_big_new.csv', index=False, sep=';')'''

"data = pd.read_csv(f'./train_big.csv', sep=';')\nnew = pd.read_csv(f'./my_data.csv', sep=';')\ndata = pd.concat([data, new], ignore_index=True)\ndata.to_csv('train_big_new.csv', index=False, sep=';')"

In [10]:
'''data = pd.read_csv("./train.csv", sep=';')
new_br = pd.read_csv("./brands_pyaterochka_with_annotation.csv", sep=';')
res = pd.concat([data, new_br], ignore_index=True)
res.to_csv('./train_filled.csv', sep=';', index=False)'''

'data = pd.read_csv("./train.csv", sep=\';\')\nnew_br = pd.read_csv("./brands_pyaterochka_with_annotation.csv", sep=\';\')\nres = pd.concat([data, new_br], ignore_index=True)\nres.to_csv(\'./train_filled.csv\', sep=\';\', index=False)'

In [11]:
def clean_labels(labels):
    """
    Заменяет русскую букву 'В' на латинскую 'B' и корректирует некорректные метки 'B-O' и 'I-O'.
    """
    cleaned = []
    for lab in labels:
        lab = lab.replace("В-", "B-").replace("В", "B")
        if lab in ("B-O", "I-O"):
            lab = "O"
        cleaned.append(lab)
    return cleaned


def load_and_convert_data(file_path):
    """
    Загружает CSV с колонками 'sample' и 'annotation',
    конвертирует VOLUME/PERCENT→O и чистит метки.
    """
    try:
        df = pd.read_csv(file_path, sep=';')
    except:
        try:
            df = pd.read_csv(file_path, sep=',')
        except:
            df = pd.read_csv(file_path)

    records = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="🔄 Converting"):
        text = row['sample']
        ann = row['annotation']
        try:
            ann = ast.literal_eval(ann) if isinstance(ann, str) else ann
        except:
            continue

        converted = []
        for start, end, label in ann:
            if label.startswith(('B-VOLUME','I-VOLUME','B-PERCENT','I-PERCENT')):
                new_label = 'O'
            else:
                new_label = label
            converted.append((start, end, new_label))

        raw = [lab for _, _, lab in converted]
        cleaned = clean_labels(raw)
        conv = [(s, e, cleaned[i]) for i, (s, e, _) in enumerate(converted)]
        records.append({'sample': text, 'annotation': conv})

    df_out = pd.DataFrame(records)
    print(f"📊 Loaded and converted: {len(df_out):,} samples")
    return df_out


def balance_by_duplication_df(
    df,
    annotation_col='annotation',
    min_count=2000,
    max_dup=10,
    max_dup_per_row=5,
    random_seed=42,
    min_brand_examples=3
):
    """
    Балансирует за счёт дублирования редких меток,
    но не более max_dup_per_row дубликатов на строку.
    """
    random.seed(random_seed)
    label_counts = Counter()
    for ann in df[annotation_col]:
        labels = {lab for _, _, lab in ann if lab != 'O'}
        for lab in labels:
            label_counts[lab] += 1

    balanced_indices = []
    dup_counter = Counter()

    # индивидуальная проверка каждой строки
    for idx, row in df.iterrows():
        ann = row[annotation_col]
        labels = {lab for _, _, lab in ann if lab != 'O'}
        if not labels:
            balanced_indices.append(idx)
            continue
        freqs = [label_counts[lab] for lab in labels]
        # если редкая метка — добавляем, но с ограничением
        if min(freqs) < min_count:
            times = min(max_dup, max_dup_per_row)
        else:
            times = 1
        for _ in range(times):
            if dup_counter[idx] < max_dup_per_row:
                balanced_indices.append(idx)
                dup_counter[idx] += 1

    balanced_df = df.loc[balanced_indices].reset_index(drop=True)
    print(f"📊 Before balance: {len(df):,}, After: {len(balanced_df):,}")
    return balanced_df


def stratified_split_by_entities(df, test_size=0.2, random_state=42, max_train_frac=0.9):
    """
    Разбивает на train/val, следя за тем,
    чтобы ни один бренд не преобладал больше, чем max_train_frac в train.
    """
    df = df.copy().reset_index(drop=True)
    rng = np.random.RandomState(random_state)
    idxs = np.arange(len(df))
    train_idx, val_idx = train_test_split(
        idxs, test_size=test_size, random_state=rng.randint(1e6)
    )
    train_set, val_set = set(train_idx), set(val_idx)

    # собираем по брендам
    brand_to_idxs = defaultdict(list)
    for i, row in df.iterrows():
        for s, e, lbl in row['annotation']:
            if 'BRAND' in lbl:
                brand = row['sample'][s:e]
                brand_to_idxs[brand].append(i)

    # убеждаемся, что ни один бренд не слишком концентрируется в train
    for brand, ids in brand_to_idxs.items():
        train_ids = [i for i in ids if i in train_set]
        if len(train_ids) / max(1, len(ids)) > max_train_frac:
            # перемещаем избыток в val
            excess = len(train_ids) - int(max_train_frac * len(ids))
            for i in rng.choice(train_ids, size=excess, replace=False):
                train_set.remove(i)
                val_set.add(i)

    train_df = df.loc[sorted(train_set)].reset_index(drop=True)
    val_df   = df.loc[sorted(val_set)].reset_index(drop=True)
    print(f"📊 Train: {len(train_df)}, Validation: {len(val_df)}")
    return train_df, val_df


if __name__ == "__main__":
    df_conv = load_and_convert_data("./train_big_new.csv")

    df_bal = balance_by_duplication_df(
        df_conv,
        max_dup=6,
        min_count=2000,
        max_dup_per_row=3,      # не более 3 дубликатов на строку
        min_brand_examples=3
    )

🔄 Converting: 100%|██████████| 39281/39281 [00:09<00:00, 4186.33it/s]


📊 Loaded and converted: 39,163 samples
📊 Before balance: 39,163, After: 39,177


In [12]:
train, val = stratified_split_by_entities(
        df_bal,
        test_size=0.01,
        max_train_frac=0.9      # ни один бренд не займет более 90% в train
    )

📊 Train: 34185, Validation: 4992


In [13]:
train[train['sample'].str.contains("милка")]

,sample,annotation
3418,милка,"[(0, 5, B-BRAND)]"
18725,томилка,"[(0, 7, B-TYPE)]"
25396,шоколадная паста милка,"[(0, 10, B-TYPE), (11, 16, I-TYPE), (17, 22, B..."


In [14]:
val[val['sample'].str.contains("милка")]

,sample,annotation
4263,шоколад милка,"[(0, 7, B-TYPE), (8, 13, B-BRAND)]"
4973,шоколад молочный милка,"[(0, 8, B-TYPE), (9, 17, I-TYPE), (18, 23, B-B..."


In [15]:
train

,sample,annotation
0,музыкальные инструменты,"[(0, 11, B-TYPE), (12, 23, I-TYPE)]"
1,hrtx,"[(0, 4, O)]"
2,сельдер,"[(0, 7, B-TYPE)]"
3,сливи,"[(0, 5, B-TYPE)]"
4,тнеска,"[(0, 6, B-TYPE)]"
...,...,...
34180,паста шоколадно-молочная шарлиз 350,"[(0, 5, B-TYPE), (6, 24, B-TYPE), (25, 31, B-B..."
34181,пастила белевская кедровым,"[(0, 7, B-TYPE), (8, 17, B-BRAND), (18, 26, B-..."
34182,пастила шармэль ароматом ванили,"[(0, 7, B-TYPE), (8, 15, B-BRAND), (16, 24, B-..."
34183,печенье belvita утреннее,"[(0, 7, B-TYPE), (8, 15, B-BRAND), (16, 24, B-..."


In [16]:
train.groupby(by=['sample']).count().sort_values(by='annotation')

,annotation
sample,
приправа для мяс,1
приправа для цезаря,1
приправа для утки,1
приправа для у,1
приправа для солянки,1
...,...
ahmad tea,12
red bull,12
сады придонья,12


In [17]:
val.groupby(by=['sample']).count().sort_values(by='annotation')

,annotation
sample,
молоко северная долина,1
мороженое магнат трюфель,1
мороженное большой папа,1
морковь натуров,1
морковь белоручк,1
...,...
global village selection,3
креветки красная цена,3
вода заповедник здоровья,3


In [18]:
train[train['sample'] == 'красный октябрь']

,sample,annotation
1248,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"
5766,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"
10911,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"
12242,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"
12456,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"
13762,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"
15180,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"
20445,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"
27544,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"
30722,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"


In [19]:
val[val['sample'] == 'красный октябрь']

,sample,annotation
838,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"
1020,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"
3493,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"


## 🔤 4. Токенизация и выравнивание меток

In [20]:
def tokenize_and_align_labels_word_level(example, tokenizer, label_to_id, max_length=128):
    text = example['sample']
    annotations = example['annotation']

    # вычисление позиций слов
    words = text.split()
    word_positions = []
    i = 0
    for w in words:
        while i < len(text) and text[i].isspace():
            i += 1
        start = text.find(w, i)
        end = start + len(w)
        word_positions.append((start, end))
        i = end

    # выравнивание аннотаций на уровне слов
    word_labels = ['O'] * len(words)
    for s, e, lbl in sorted(annotations, key=lambda x: x[0]):
        if lbl.startswith(('B-VOLUME','I-VOLUME','B-PERCENT','I-PERCENT')):
            continue
        # ищем перекрытие
        overlaps = [
            (i, min(e, we) - max(s, ws))
            for i,(ws,we) in enumerate(word_positions)
            if max(s, ws) < min(e, we)
        ]
        if overlaps:
            idx, _ = max(overlaps, key=lambda x: x[1])
            word_labels[idx] = lbl

    # токенизация с паддингом и транкацией
    tokenized = tokenizer(
        words,
        is_split_into_words=True,
        padding="max_length",
        truncation=True,
        max_length=config.MAX_LENGTH,
        return_attention_mask=True,
        return_token_type_ids=False
    )

    # выравнивание меток на уровне токенов
    word_ids = tokenized.word_ids()
    aligned_labels = []
    prev_word_idx = None

    for word_idx in word_ids:
        if word_idx is None:
            aligned_labels.append(-100)
        elif word_idx != prev_word_idx:
            lbl = word_labels[word_idx]
            if lbl.startswith('I-') and prev_word_idx is None:
                lbl = 'B-'+lbl.split('-',1)[1]
            aligned_labels.append(config.LABEL_TO_ID.get(lbl, config.LABEL_TO_ID['O']))
            prev_word_idx = word_idx
        else:
            lbl = word_labels[word_idx]
            if lbl.startswith('B-'):
                lbl = 'I-'+lbl.split('-',1)[1]
            aligned_labels.append(config.LABEL_TO_ID.get(lbl, config.LABEL_TO_ID['O']))

    tokenized["labels"] = aligned_labels
    return tokenized

In [21]:
def prepare_datasets_word_level(train_df, val_df, tokenizer, config, test_size=0.2, random_state=42):
    """
    Подготовка Dataset с word-level токенизацией, где tokenize_and_align_labels_word_level
    вызывается для каждого примера по отдельности.
    """
    print(f"📊 Train: {len(train_df)}, Validation: {len(val_df)}")

    # 2. Функция-обертка для одного примера
    def tokenize_example(example):
        # example["sample"] — строка текста
        # example["annotation"] — список аннотаций (start, end, label)
        single = {"sample": example["sample"], "annotation": example["annotation"]}
        tokenized = tokenize_and_align_labels_word_level(
            single, tokenizer, config.LABEL_TO_ID, config.MAX_LENGTH
        )
        # возвращаем dict с полями input_ids, attention_mask, labels
        return tokenized

    # 3. Токенизация train и val по примерам
    print("🔄 Токенизация training данных...")
    train_records = [tokenize_example(row) for _, row in train_df.iterrows()]

    print("🔄 Токенизация validation данных...")
    val_records = [tokenize_example(row) for _, row in val_df.iterrows()]

    # 4. Создание HuggingFace Dataset
    train_dataset = Dataset.from_list(train_records)
    val_dataset = Dataset.from_list(val_records)

    return {"train": train_dataset, "validation": val_dataset}


## 🧠 5. BERT+Парсер объмов

In [22]:
import re

class SoftValidationParser:
    def __init__(self):
        self.volume_units = [
            'г', 'гр\\.?', 'грамм\\.?', 'кг', 'килограмм\\.?',
            'мл', 'мл\\.?', 'миллилитр\\.?', 'л', 'л\\.?', 'литр\\.?',
            'литра\\.?', 'шт\\.?', 'штук\\.?'
        ]
        self.percent_units = ['%', 'процент(ов)?', 'проц\\.?', 'жирн\\.?']

        # шаблоны
        self.vol_pattern = re.compile(
            rf'(\d+[.,]?\d*)\s*({"|".join(self.volume_units)})',
            flags=re.IGNORECASE
        )
        self.pct_pattern = re.compile(
            rf'(\d+[.,]?\d*)\s*({"|".join(self.percent_units)})',
            flags=re.IGNORECASE
        )
        # число в конце строки
        self.plain_number_pattern = re.compile(r'(\d+[.,]?\d*)$')

    def preprocess_volume_text(self, text):
        replacements = {
            r'(\d+(?:[.,]\d+)?)\s*([а-яА-Я%]+)': r'\1 \2',
            r'(\d+),(\d+)': r'\1.\2',
            r'\b(грамм?|гр\.?)\b': 'г',
            r'\b(килограмм?|кило|кг\.?)\b': 'кг',
            r'\b(миллилитр?|мл\.?)\b': 'мл',
            r'\b(литр?|л\.?)\b': 'л',
            r'\b(штук[аи]?|шт\.?)\b': 'шт',
            r'\bпол\b': '0.5',
            r'\bполтора\b': '1.5',
            r'\bполовин[ау]\b': '0.5',
            r'\bчетверть\b': '0.25',
            r'\b(\d+(?:[.,]\d+)?)\s*%\b': r'\1 %',
            r'\b(\d+(?:[.,]\d+)?)\s*(процент(?:ов)?|проц\.?|жирн\.?|крепост(?:ь)?|градус(?:ов)?)\b': r'\1 %',
        }
        result = text
        for pat, repl in replacements.items():
            result = re.sub(pat, repl, result, flags=re.IGNORECASE)
        return result

    def parse_with_soft_validation(self, text, type_entity=None):
        normalized = self.preprocess_volume_text(text)
        entities = []

        # 1. Поиск объёмов с единицами
        for m in self.vol_pattern.finditer(normalized):
            start, end = m.span()
            parts = normalized[start:end].split()
            offset = start
            for i, part in enumerate(parts):
                p_start = normalized.find(part, offset)
                p_end = p_start + len(part)
                tag = 'B-VOLUME' if i == 0 else 'I-VOLUME'
                entities.append((p_start, p_end, tag))
                offset = p_end

        # 2. Поиск процентов
        for m in self.pct_pattern.finditer(normalized):
            start, end = m.span()
            parts = normalized[start:end].split()
            offset = start
            for i, part in enumerate(parts):
                p_start = normalized.find(part, offset)
                p_end = p_start + len(part)
                tag = 'B-PERCENT' if i == 0 else 'I-PERCENT'
                entities.append((p_start, p_end, tag))
                offset = p_end

        # 3. Если в конце – число без единиц И до него нет признаков объёма/процента
        m = self.plain_number_pattern.search(normalized)
        if m:
            start, end = m.span(1)
            prefix_text = normalized[:start]
            # проверяем отсутствие любых вхождений объёмов/процентов перед этим числом
            if not self.vol_pattern.search(prefix_text) and not self.pct_pattern.search(prefix_text):
                # и число не пересекает уже найденные сущности
                if not any(s < end and e > start for s, e, _ in entities):
                    entities.append((start, end, 'B-VOLUME'))

        entities.sort(key=lambda x: x[0])
        return entities

In [23]:
label_list = ["O", "B-TYPE", "I-TYPE", "B-BRAND", "I-BRAND"]
label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for l, i in label2id.items()}

# Исправленная версия экстрактора с решением всех трех проблем
class ImprovedProductEntityExtractor:
    def __init__(self,
                 ner_model_path="./ner_ecommerce_model",
                 catalog_brands=None,
                 catalog_categories=None):

        self.tokenizer_ner = AutoTokenizer.from_pretrained(ner_model_path,
                                                           is_split_into_words=True)
        self.model_ner = AutoModelForTokenClassification.from_pretrained(ner_model_path)
        # ipeline NER с правильной агрегацией - ПРОСТАЯ СТРАТЕГИЯ
        self.pipeline_ner = pipeline(
            "ner",
            model=self.model_ner,
            tokenizer=self.tokenizer_ner,
            aggregation_strategy="simple",  # Возвращаем к simple!
            device=-1
        )

        # Инициализация остальных компонентов
        self.catalog_brands = catalog_brands or []
        self.catalog_categories = catalog_categories or []
        self.morph = pymorphy3.MorphAnalyzer()
        self.id2label = {
            0: 'O',
            1: 'B-TYPE',
            2: 'I-TYPE',
            3: 'B-BRAND',
            4: 'I-BRAND'
        }

        # Парсер объемов
        self.soft_parser = SoftValidationParser()

    def lemmatize(self, word):
        """Лемматизация через pymorphy3"""
        if not word or not isinstance(word, str):
            return word or ""
        try:
            parsed = self.morph.parse(word)
            return parsed[0].normal_form if parsed else word.lower()
        except:
            return word.lower()

    def _postprocess_type_brand(self, annotations, text):
        """Исправленная постобработка TYPE/BRAND с правильными BIO-метками"""
        tokens = text.split()

        # Если два span подряд покрывают одно слово – оба меняем на TYPE
        if (len(annotations) == 2 and
            annotations[0][2] == 'B-TYPE' and annotations[1][2] == 'B-BRAND' and
            annotations[0][1] == annotations[1][0]):
            start, _, _ = annotations[0]
            _, end, _ = annotations[1]
            return [(start, end, 'B-TYPE')]

        # Если только первое слово размечено как B-TYPE, а слов больше — отмечаем все как TYPE
        if (len(annotations) == 1 and
            annotations[0][2] == 'B-TYPE' and
            len(tokens) > 1):

            spans = []
            char_pos = 0

            for i, token in enumerate(tokens):
                token_start = char_pos
                token_end = char_pos + len(token)

                # Первый токен - B-TYPE, остальные - I-TYPE
                if i == 0:
                    spans.append((token_start, token_end, 'B-TYPE'))
                else:
                    spans.append((token_start, token_end, 'I-TYPE'))

                # Переходим к следующему токену (учитываем пробел)
                char_pos = token_end + 1

            return spans

        # Если аннотация покрывает только часть первого слова, расширяем на все слова
        if (len(annotations) == 1 and
            annotations[0][2] == 'B-TYPE' and
            len(tokens) > 0):

            # Проверяем, покрывает ли аннотация весь первый токен
            first_token_end = len(tokens[0])
            if annotations[0][1] < first_token_end and len(tokens) > 1:
                spans = []
                char_pos = 0

                for i, token in enumerate(tokens):
                    token_start = char_pos
                    token_end = char_pos + len(token)

                    if i == 0:
                        spans.append((token_start, token_end, 'B-TYPE'))
                    else:
                        spans.append((token_start, token_end, 'I-TYPE'))

                    char_pos = token_end + 1

                return spans

        return annotations

    def _map_entities_to_original_words(self, ner_results, original, corrected):
        """
        КЛЮЧЕВАЯ ФУНКЦИЯ: Правильное сопоставление NER результатов с исходными словами
        """
        if not ner_results:
            return []

        original_tokens = original.split()
        corrected_tokens = corrected.split()

        # Создаем карту: позиция в corrected -> позиция в original
        entities = []

        for ner_ent in ner_results:
            start_corr = ner_ent['start']
            end_corr = ner_ent['end']
            entity_text = corrected[start_corr:end_corr]
            label = f"B-{ner_ent['entity_group'].upper()}"

            # Находим соответствующие токены в corrected
            char_pos = 0
            found_token_indices = []

            for i, token in enumerate(corrected_tokens):
                token_start = char_pos
                token_end = char_pos + len(token)

                # Если токен пересекается с entity
                if start_corr < token_end and end_corr > token_start:
                    found_token_indices.append(i)

                char_pos = token_end + 1  # +1 для пробела

            # Мапим найденные токены на исходный текст
            if found_token_indices:
                # Берем границы от первого до последнего найденного токена
                first_token_idx = found_token_indices[0]
                last_token_idx = found_token_indices[-1]

                # Вычисляем позиции в исходном тексте
                orig_start = sum(len(original_tokens[i]) + 1 for i in range(first_token_idx))
                orig_end = orig_start + sum(len(original_tokens[i]) + 1 for i in range(first_token_idx, last_token_idx + 1)) - 1

                # Корректируем границы
                if orig_start < len(original):
                    if orig_end > len(original):
                        orig_end = len(original)
                    entities.append((orig_start, orig_end, label))

        return entities

    def extract_entities_with_soft_validation(self, query: str):
        """
        Унифицированный метод, который сначала лемматизирует текст,
        затем вызывает predict() для получения BIO-разметки на уровне слов,
        и дополнительно возвращает raw-список для дальнейшей обработки.
        """
        # 1. Spell correction / лемматизация
        tokens = query.split()
        corrected = " ".join(self.lemmatize(tok) for tok in tokens)

        # 2. Получаем BIO-аннотации на уровне слов
        word_entities = self.predict(query)

        # 3. Собираем полный результат
        return {
            "original_query": query,
            "corrected_query": corrected,
            "annotations": word_entities,
            "type":   self._get_entity_text_list(word_entities, query, 'TYPE'),
            "brand":  self._get_entity_text_list(word_entities, query, 'BRAND'),
            "volumes":  [e for e in word_entities if e['entity'].endswith('-VOLUME')],
            "percents": [e for e in word_entities if e['entity'].endswith('-PERCENT')]
        }

    def _get_entity_text_list(self, entities, text, entity_type):
        """Возвращает список вхождений entity_type из annotations."""
        values = []
        for ent in entities:
            if ent['entity'].endswith(f"-{entity_type}"):
                values.append(text[ent['start_index']:ent['end_index']])
        return values

    def predict(self, text: str):
        """
        Для каждого слова возвращает BIO-метки:
        – если слово попало под soft-parser, используем эти метки;
        – elif слово попало под NER-модель, используем эти метки;
        – иначе 'O'.
        Возвращает кортежи (start, end, BIO).
        """
        # 1. Разбиваем на слова и считаем их spans
        words = text.split()
        spans = []
        pos = 0
        for w in words:
            spans.append((pos, pos + len(w)))
            pos += len(w) + 1

        # 2. Получаем сырые результаты
        ner_results = list(self.pipeline_ner(text, aggregation_strategy="simple"))
        soft_results = self.soft_parser.parse_with_soft_validation(text)

        # 3. Инициализируем группы для каждого слова
        word_groups = [None] * len(spans)

        # 4. Применяем soft-parser (они уже содержат B-/I- метки, берём просто группу)
        for start, end, label in soft_results:
            grp = label.split('-', 1)[1]
            for idx, (w_start, w_end) in enumerate(spans):
                if start < w_end and end > w_start:
                    word_groups[idx] = grp

        # 5. Применяем NER туда, где ещё нет soft-метки
        for r in ner_results:
            grp = r["entity_group"].upper()
            for idx, (w_start, w_end) in enumerate(spans):
                if word_groups[idx] is None and r["start"] < w_end and r["end"] > w_start:
                    word_groups[idx] = grp

        # 6. Строим BIO-метки по последовательности групп
        bio_tags = []
        prev_group = None
        for grp in word_groups:
            if grp is None:
                bio_tags.append('O')
                prev_group = None
            else:
                if grp == prev_group:
                    bio_tags.append(f'I-{grp}')
                else:
                    bio_tags.append(f'B-{grp}')
                    prev_group = grp

        # 6.1 Дополнительная пост-коррекция:
        # если слово из 1 буквы, метка B-TYPE или I-TYPE,
        # и до него и после него нет BRAND, ставим 'O'
        corrected_tags = bio_tags.copy()
        for i, (tag, (start, end)) in enumerate(zip(bio_tags, spans)):
            word = text[start:end]
            if len(word) == 1 and tag.endswith('-TYPE'):
                prev_is_brand = i > 0 and bio_tags[i-1].endswith('-BRAND')
                next_is_brand = i < len(bio_tags)-1 and bio_tags[i+1].endswith('-BRAND')
                if not prev_is_brand and not next_is_brand:
                    corrected_tags[i] = 'O'

        # 7. Возвращаем результат с учётом скорректированных тегов
        return [
            (spans[i][0], spans[i][1], corrected_tags[i])
            for i in range(len(spans))
        ]


    def _remove_overlaps(self, entities):
        """Удаляет перекрывающиеся сущности, оставляя более точные"""
        if not entities:
            return []

        sorted_entities = sorted(entities, key=lambda x: (x[0], x[1] - x[0]))
        result = []

        for current in sorted_entities:
            overlapped = False
            for i, existing in enumerate(result):
                if (current[0] < existing[1] and current[1] > existing[0]):  # есть пересечение
                    # Оставляем более короткую (более точную) сущность
                    if (current[1] - current[0]) <= (existing[1] - existing[0]):
                        result[i] = current
                    overlapped = True
                    break

            if not overlapped:
                result.append(current)

        return result

    def _get_entity_text(self, entities, text, entity_type):
        """Извлекает текст сущности определенного типа"""
        for start, end, label in entities:
            if entity_type in label:
                return text[start:end]
        return None

    def _format_volume(self, entity, text):
        """Форматирует сущность объема"""
        start, end, label = entity
        return {
            'value': text[start:end],
            'original': text[start:end],
            'position': (start, end)
        }

    def _format_percent(self, entity, text):
        """Форматирует сущность процента"""
        start, end, label = entity
        return {
            'value': text[start:end],
            'original': text[start:end],
            'position': (start, end)
        }

    # Алиас для обратной совместимости
    def extract_entities(self, query):
        return self.extract_entities_with_soft_validation(query)

In [24]:
def compute_macro_f1_metrics(pred_entities, true_entities):
    """
    pred_entities: list of lists of (start, end, label) предсказаний для каждого примера
    true_entities: list of lists of (start, end, label) эталонных сущностей для каждого примера
    Возвращает dict с macro_f1, macro_precision, macro_recall.
    """
    # Инициализируем счётчики TP, FP, FN по каждому типу
    stats = {
        'TYPE': {'TP': 0, 'FP': 0, 'FN': 0},
        'BRAND': {'TP': 0, 'FP': 0, 'FN': 0},
        'O': {'TP': 0, 'FP': 0, 'FN': 0},
    }
    # Функция для получения типа без префикса B- или I-
    def base_type(lbl):
        return lbl.split('-')[-1]

    # Для каждого примера
    for preds, trues in zip(pred_entities, true_entities):
        matched = set()
        # Ищем TP и FP
        for p in preds:
            p_span = (p[0], p[1], p[2])
            p_type = base_type(p[2])
            found_tp = False
            for i, t in enumerate(trues):
                if i in matched:
                    continue
                # Матчим span и label полностью
                if p[0] == t[0] and p[1] == t[1] and p[2] == t[2]:
                    stats[p_type]['TP'] += 1
                    matched.add(i)
                    found_tp = True
                    break
            if not found_tp:
                stats[p_type]['FP'] += 1
        # Ищем FN: истинные, которые не были заматчены
        for i, t in enumerate(trues):
            if i not in matched:
                t_type = base_type(t[2])
                stats[t_type]['FN'] += 1

    # Вычисляем precision, recall, F1 по каждому типу
    f1_sum = 0
    prec_sum = 0
    rec_sum = 0
    n_types = len(stats)
    for etype, vals in stats.items():
        TP = vals['TP']
        FP = vals['FP']
        FN = vals['FN']
        prec = TP / (TP + FP) if TP + FP > 0 else 0.0
        rec  = TP / (TP + FN) if TP + FN > 0 else 0.0
        f1   = 2 * prec * rec / (prec + rec) if prec + rec > 0 else 0.0
        prec_sum += prec
        rec_sum  += rec
        f1_sum   += f1

    macro_precision = prec_sum / n_types
    macro_recall    = rec_sum  / n_types
    macro_f1        = f1_sum   / n_types

    return {
        'macro_precision': macro_precision,
        'macro_recall':    macro_recall,
        'macro_f1':        macro_f1
    }

In [25]:
def labels_to_entities(labels):
    """
    Преобразует BIO-последовательность меток в список кортежей (start, end, label),
    где end — индекс токена сразу после последнего токена сущности.
    Метка возвращается с префиксом B- или I- для использования в compute_macro_f1_metrics.
    """
    entities = []
    start = None
    cur_label = None

    for i, tag in enumerate(labels):
        if tag.startswith("B-"):
            # закрываем предыдущую сущность
            if cur_label is not None:
                entities.append((start, i, cur_label))
            start = i
            cur_label = tag
        elif tag.startswith("I-") and cur_label == tag:
            # продолжаем сущность
            continue
        else:
            # текущий токен не входит в сущность
            if cur_label is not None:
                entities.append((start, i, cur_label))
                start = None
                cur_label = None

    # если в конце осталась открытая сущность
    if cur_label is not None:
        entities.append((start, len(labels), cur_label))

    return entities

In [26]:
def compute_metrics(p):
    predictions, labels = p
    preds = np.argmax(predictions, axis=-1)
    metric = evaluate.load("seqeval")
    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, labels)
    ]
    seqeval_results = metric.compute(predictions=true_predictions, references=true_labels)
    # Извлекаем entity-level F1 для нужных типов и считаем macro
    entity_types = ["TYPE", "BRAND"]
    # Преобразуем последовательности меток в entity spans
    pred_entities = [labels_to_entities(pred_seq) for pred_seq in true_predictions]
    true_entities = [labels_to_entities(true_seq) for true_seq in true_labels]
    # print(pred_entities, true_entities)
    # Теперь вызываем macro F1 с правильным форматом данных
    macro_metrics = compute_macro_f1_metrics(pred_entities, true_entities)
    return {
        "token_precision": seqeval_results["overall_precision"],
        "token_recall": seqeval_results["overall_recall"],
        "token_f1": seqeval_results["overall_f1"],
        "token_accuracy": seqeval_results["overall_accuracy"],
        "macro_precision": macro_metrics["macro_precision"],
        "macro_recall":    macro_metrics["macro_recall"],
        "macro_f1":        macro_metrics["macro_f1"]
    }

## 🚀 6. Обучение модели

In [27]:
# Конвертируем и токенизируем
print("=" * 70)
print("🔤 ТОКЕНИЗАЦИЯ И ВЫРАВНИВАНИЕ")
print("=" * 70)

# Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME)
print(f"📚 Токенизатор: {config.MODEL_NAME}")

# Создаем Dataset
tokenized_datasets = DatasetDict(prepare_datasets_word_level(train, val, tokenizer,
                                                             config, test_size=0.1))

print("✅ Токенизация завершена успешно!")

# Проверяем распределение меток в финальном датасете
label_counts = Counter()
for item in tokenized_datasets['train']:
    for label_id in item['labels']:
        if label_id != -100:
            label_counts[config.ID_TO_LABEL[label_id]] += 1

print("\n📊 Распределение меток в обучающем наборе:")
total_labels = sum(label_counts.values())
for label, count in label_counts.most_common():
    print(f"  {label}: {count:,} ({count/total_labels*100:.1f}%)")

🔤 ТОКЕНИЗАЦИЯ И ВЫРАВНИВАНИЕ


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

📚 Токенизатор: cointegrated/rubert-tiny2
📊 Train: 34185, Validation: 4992
🔄 Токенизация training данных...
🔄 Токенизация validation данных...
✅ Токенизация завершена успешно!

📊 Распределение меток в обучающем наборе:
  I-TYPE: 39,583 (35.8%)
  B-TYPE: 25,202 (22.8%)
  I-BRAND: 22,275 (20.1%)
  B-BRAND: 14,191 (12.8%)
  O: 9,413 (8.5%)


In [28]:
indices = list(range(10))
batch = tokenized_datasets['train'].select(indices)

for i, example in enumerate(batch):
    print(f"=== Пример {i} ===")
    input_ids = example["input_ids"]
    labels    = example["labels"]
    tokens    = tokenizer.convert_ids_to_tokens(input_ids)
    print("input_ids:", input_ids)
    print("labels:   ", labels)
    print("tokens:   ", tokens)
    print()

=== Пример 0 ===
input_ids: [2, 42472, 35426, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
labels:    [-100, 1, 2, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -10

In [29]:
train.to_csv('./train_big_str.csv', index=False, sep=';')
val.to_csv('./val_big_str.csv', index=False, sep=';')

In [ ]:
# Инициализация модели
print("=" * 70)
print("🧠 ИНИЦИАЛИЗАЦИЯ BERT+CRF МОДЕЛИ")
print("=" * 70)

tokenizer = BertTokenizerFast.from_pretrained(
            config.MODEL_NAME,
            do_lower_case=True,
            strip_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            cls_token="[CLS]",
            pad_token="[PAD]",
            mask_token="[MASK]",
            truncation=True,
            max_length=config.MAX_LENGTH
)
tokenizer.pre_tokenizer = lambda text: text.split()
# Модель
model = AutoModelForTokenClassification.from_pretrained(
    config.MODEL_NAME,
    num_labels=len(config.LABELS),
    id2label=config.ID_TO_LABEL,
    label2id=config.LABEL_TO_ID
)
# Подсчет параметров
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"🔢 Всего параметров: {total_params:,}")
print(f"🎯 Обучаемых параметров: {trainable_params:,}")
print(f"📊 Количество меток: {len(config.LABELS)}")
print(f"🏷️ Метки: {config.LABELS}")
print(f"💻 Модель размещена на: {device}")

print("\n✅ BERT+CRF модель готова!")
# Настройка Data Collator
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    label_pad_token_id=-100,
    padding=True
)

# Training Arguments
training_args = TrainingArguments(
    output_dir=config.OUTPUT_DIR,
    num_train_epochs=config.EPOCHS,
    per_device_train_batch_size=config.BATCH_SIZE,
    per_device_eval_batch_size=config.BATCH_SIZE,
    gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,
    learning_rate=config.LEARNING_RATE,
    weight_decay=config.WEIGHT_DECAY,
    warmup_ratio=config.WARMUP_RATIO,
    # Evaluation
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=400,
    # Метрики и early stopping
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    greater_is_better=True,
    # Логирование
    logging_dir='./logs',
    logging_steps=50,
    # Оптимизации
    fp16=True,  # Смешанная точность для ускорения
    dataloader_pin_memory=True,
    dataloader_num_workers=2,
    # Сохранение
    save_total_limit=3,
    # Отключаем внешнее логирование
    report_to='none',
    # Для воспроизводимости
    seed=42,
    save_safetensors=False,

)

# Инициализируем Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print("=" * 70)
print("🚀 НАЧИНАЕМ ОБУЧЕНИЕ BERT+CRF МОДЕЛИ")
print("=" * 70)
print(f"📊 Train образцов: {len(tokenized_datasets['train']):,}")
print(f"🔍 Val образцов: {len(tokenized_datasets['validation']):,}")
print(f"⏰ Эпох: {config.EPOCHS}")
print(f"📈 BERT Learning rate: {config.LEARNING_RATE}")
print(f"🔗 CRF Learning rate: {config.CRF_LEARNING_RATE}")
print(f"📦 Batch size: {config.BATCH_SIZE}")
print(f"🎯 Цель: macro-F1 ≥ 0.8")
print(f"💻 Устройство: {device}")
print("=" * 70)

# Запускаем обучение
print("\n🏋️‍♂️ Начинаем обучение...")
print("(Это может занять 20-40 минут в зависимости от GPU)")

try:
    # Обучение
    train_result = trainer.train()

    print("\n🎉 Обучение завершено!")
    print(f"📈 Финальный training loss: {train_result.training_loss:.4f}")

    # Сохранение модели
    tokenizer.save_pretrained(config.OUTPUT_DIR)
    trainer.save_model(config.OUTPUT_DIR)

    print(f"💾 Модель сохранена в {config.OUTPUT_DIR}'")

except KeyboardInterrupt:
    print("\n⚠️ Обучение прервано пользователем")
    print("📁 Частично обученная модель сохранена")

except Exception as e:
    print(f"\n❌ Ошибка во время обучения: {e}")
    print("🔍 Проверьте данные и конфигурацию")

print("\n" + "=" * 70)

🧠 ИНИЦИАЛИЗАЦИЯ BERT+CRF МОДЕЛИ


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🔢 Всего параметров: 29,097,677
🎯 Обучаемых параметров: 29,097,677
📊 Количество меток: 5
🏷️ Метки: ['O', 'B-TYPE', 'I-TYPE', 'B-BRAND', 'I-BRAND']
💻 Модель размещена на: cuda

✅ BERT+CRF модель готова!
🚀 НАЧИНАЕМ ОБУЧЕНИЕ BERT+CRF МОДЕЛИ
📊 Train образцов: 34,185
🔍 Val образцов: 4,992
⏰ Эпох: 15
📈 BERT Learning rate: 3e-05
🔗 CRF Learning rate: 0.0001
📦 Batch size: 32
🎯 Цель: macro-F1 ≥ 0.8
💻 Устройство: cuda

🏋️‍♂️ Начинаем обучение...
(Это может занять 20-40 минут в зависимости от GPU)


Step,Training Loss,Validation Loss,Token Precision,Token Recall,Token F1,Token Accuracy,Macro Precision,Macro Recall,Macro F1
200,1.456700,1.498223,0.109385,0.123114,0.115844,0.267464,0.310562,0.144113,0.151289
400,0.980900,1.134794,0.254342,0.277902,0.265600,0.467241,0.431230,0.317042,0.258025
600,0.695800,0.886547,0.479383,0.482893,0.481131,0.599888,0.446280,0.410938,0.397516
800,0.630800,0.687307,0.551457,0.596444,0.573069,0.686109,0.468833,0.471150,0.455119
1000,0.558100,0.579844,0.619061,0.667713,0.642467,0.749768,0.513115,0.515137,0.506565
1200,0.457200,0.521187,0.676684,0.723293,0.699213,0.783859,0.524591,0.540250,0.524336
1400,0.449700,0.493501,0.706772,0.746900,0.726282,0.801339,0.535848,0.555371,0.536504
1600,0.399600,0.430906,0.729100,0.780517,0.753933,0.829294,0.555758,0.582316,0.562505
1800,0.407000,0.516998,0.731534,0.753175,0.742197,0.802269,0.544201,0.563825,0.541273
2000,0.382100,0.310393,0.817603,0.843867,0.830527,0.880617,0.582799,0.600056,0.587817


In [ ]:
from google.colab import runtime
runtime.unassign()

## 🧪 8. Тестирование и демонстрация

In [ ]:
def test_model_all_word_labels(config, device='cpu'):
    """
    Тестирование модели с выводом меток для всех слов.
    """
    # Загружаем экстрактор
    extractor = ImprovedProductEntityExtractor(config.OUTPUT_DIR)
    test_texts = [
        'молоко без сахара', 'j7 сок',
              'сметана домик 7 %', 'конфеты m&ms',
              'колбас красна цена', 'торог',
              'кола 2л', 'кола 2 литра',
              'сметана домик 7% 2 литра', 'пиво хейнекен',
              'печенье милка', 'шоколад россия',
              'шоколад alpen', 'сметана домик 7',
              'сок 5 литра', 'напиток fresh тропический',
              'красная цена кукуруза', 'белоручка реди',
              'масло сливочное экомилк', 'сливки 33 %', 'сливки для взбиванмя',
              'сок сады придонья 2 литра', 'сосиски из курицы',
              'соус длч спагетти', 'сулугуни киржачский',
              'сок 5 × 0,33 л', 'сок 5 по 0,33 л',
              'кеф 3%', 'кифири danone 1,5 %', 'три процента',
              'жопа в деревне', 'йогурт в деревне', 'йогурт домик',
              'шоколад альпин голд', 'сыр с', 'молоко пармалат 5 % 500мл'
    ]

    print("🧪 Тестирование с метками для каждого слова:")
    print("=" * 80)

    for i, text in enumerate(test_texts, 1):
        print(f"\n{i}. Текст: '{text}'")

        try:
            # Получаем метки для всех слов (включая постпроцессинг)
            all_word_labels = extractor.predict(text)

            # Форматируем вывод как в вашем примере
            result_string = f"{text}; {all_word_labels}"
            print(f"   Результат: {result_string}")

            # Дополнительно показываем каждое слово с меткой
            print("   Детализация:")
            words = text.split()
            for j, ((start, end, label), word) in enumerate(zip(all_word_labels, words)):
                emoji = {'TYPE': '📦', 'BRAND': '🏷️', 'VOLUME': '⚖️', 'PERCENT': '📊', 'O': '⚪'}.get(label, '🔖')
                print(f"     {emoji} '{word}' → ({start}, {end}, '{label}')")

        except Exception as e:
            print(f"   ❌ Ошибка: {e}")

    print("\n" + "=" * 80)


In [ ]:
# Загрузка обученной модели
print("=" * 70)
print("🔮 ТЕСТИРОВАНИЕ ОБУЧЕННОЙ МОДЕЛИ")
print("=" * 70)
# Тестирование
test_model_all_word_labels(config, device)